In [112]:
import numpy as np
import tensorflow as tf
import torch as th
from spektral.layers.convolutional import ARMAConv as ARMAConvSpektral
from torch_geometric.nn import ARMAConv as ARMAConvTorch

In [5]:
a_input = np.load('adjacency_df.npy')

In [25]:
def array_matrices_to_array_edge_index(array_matrices: np.ndarray):
    print(array_matrices.shape)
    array_edge_indices = []
    array_edge_indices_weighted = []

    # Iterate over the adjacency matrices
    for matrix in array_matrices:
        edge_indices = []
        edge_weights = []
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if matrix[i, j] != 0:
                    # Add the edge indices and the weight
                    edge_indices.append([i, j])
                    edge_weights.append(matrix[i, j])

        #the edge indices and weights shuld always contain 3 columns, fill with [] if not

        edge_indices = np.array(edge_indices)
        edge_weights = np.array(edge_weights)
        array_edge_indices.append(edge_indices)
        array_edge_indices_weighted.append(edge_weights)

    return np.array(array_edge_indices), np.array(array_edge_indices_weighted)

In [29]:
array_matrices_to_array_edge_index(a_input[1:2])

(1, 3, 3)


(array([[[0, 1],
         [1, 0]]]),
 array([[1., 1.]]))

In [133]:
node_features = th.tensor([
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
])

edge_indices = th.tensor(
    [
        [0, 1, 1, 1],  # Source nodes
        [0, 2, 0, 1],  # Target nodes
    ]

)

data = PairData(x=node_features, edge_index=edge_indices)

# Adjacency matrix
# [1., 3., 0.],
# [3., 4., 2.],
# [0., 2., 0.],


edge_weights = th.tensor([
    1., 2., 3., 4.
])

print(node_features.shape)
print(edge_indices.shape)
print(edge_weights.shape)

arma = ARMAConvTorch(3, 3, shared_weights=True)
arma(node_features, edge_indices, edge_weight=edge_weights)

torch.Size([3, 3])
torch.Size([2, 4])
torch.Size([4])


tensor([[0.0000, 2.1091, 0.0000],
        [0.0000, 2.1091, 0.0000],
        [0.4610, 0.7938, 0.0000]], grad_fn=<MeanBackward1>)

In [191]:
# node_features = tf.convert_to_tensor([
#     [
#         [1., 0., 0.],
#         [0., 0., 0.],
#         [0., 0., 0.],
#     ],
#     [
#         [0., 0., 0.],
#         [0., 2., 0.],
#         [0., 0., 0.],
#     ]
# ]
# 
# )
# 
adjacency_graph = tf.convert_to_tensor(
    [
        [
            [1., 0., 0.],
            [0., 1., 0.],
            [0., 0., 1.]
        ],
        [
            [0., 2., 3.,5.],
            [2., 0., 4.,0.],
            [3., 4., 0.,0.],
            [5., 4., 0.,0.],
        ],
    ]
)

node_features = tf.convert_to_tensor(
    [
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
    ]
)

adjacency_graph = tf.convert_to_tensor(

    [
        [1., 3., 0.],
        [3., 4., 2.],
        [0., 2., 0.],
    ],

)

print(node_features.shape)
print(adjacency_graph.shape)

arma = ARMAConvSpektral(node_features.shape[-1], shared_weights=True)
arma([node_features, adjacency_graph])

SyntaxError: invalid syntax (293872022.py, line 18)

In [185]:
import torch
from torch_geometric.nn import ARMAConv
from torch_geometric.data import Data

# Sample data preparation
num_nodes = 10
num_node_features = 5
num_classes = 3
edges = torch.tensor([[0, 1, 1, 2],
                      [1, 2, 0, 3]], dtype=torch.long)
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])

# Graph data object
data = Data(x=node_features, edge_index=edges)


# Model
class PyGARMA(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(num_node_features, num_classes, num_stacks=1, num_layers=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        return x


model_pyg = PyGARMA()
model_pyg.train()
out = model_pyg(data)
print(out)

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281]], grad_fn=<MeanBackward1>)


In [181]:
import numpy as np
from spektral.layers import ARMAConv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from spektral.data import Graph

# Converting PyTorch data to Spektral graph
x = np.array(node_features)
a = np.zeros((num_nodes, num_nodes))
a[edges[0], edges[1]] = 1

graph = Graph(x=x, a=a)


# Spektral model
class SpektralARMA(Model):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(channels=num_classes, n_stacks=1, n_layers=1)

    def call(self, inputs):
        x, a = inputs
        return self.arma([x, a])


x_in = Input(shape=(num_node_features,))
a_in = Input((num_nodes,))
model_spektral = SpektralARMA()
model_spektral([x_in, a_in])
model_spektral.compile()
out = model_spektral([graph.x, graph.a])
print(out)


tf.Tensor(
[[  0.          0.        143.30136  ]
 [  0.          0.        147.9425   ]
 [  0.          2.176948    9.282283 ]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]], shape=(10, 3), dtype=float32)


/Users/vitor/Desktop/mestrado/venv/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [187]:
import torch

# Example adjacency matrices as dense tensors
adj_matrices = torch.tensor([
    [[1., 0., 0.],
     [0., 1., 1.],
     [0., 1., 1.]],

    [[0., 1., 0.],
     [1., 0., 1.],
     [0., 1., 0.]]
])

# Convert adjacency matrices to edge_index format and edge_type
num_nodes = adj_matrices.size(1)
edge_index = []
edge_type = []

for i, adj in enumerate(adj_matrices):
    row, col = torch.where(adj > 0)
    edge_index.append(torch.stack([row, col], dim=0))
    edge_type.append(torch.full((row.size(0),), i, dtype=torch.long))

# Concatenate all edges
edge_index = torch.cat(edge_index, dim=1)
edge_type = torch.cat(edge_type)

print(edge_index)
print(edge_type)


tensor([[0, 1, 1, 2, 2, 0, 1, 1, 2],
        [0, 1, 2, 1, 2, 1, 0, 2, 1]])
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1])
